In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import wasserstein_distance

# Load the two datasets (MCI and NC) from CSV files
file_mci = "/content/drive/MyDrive/radiomic_features_AD_dec25.csv"  # Replace with your actual file path
file_nc = "/content/drive/MyDrive/radiomic_features_nc_oct10 .csv"    # Replace with your actual file path

data_mci = pd.read_csv(file_mci)
data_nc = pd.read_csv(file_nc)

def extract_number(array_string):
    if isinstance(array_string, str) and 'array' in array_string:
        return float(array_string.split('(')[1].split(')')[0])
    else:
        return array_string

# Apply the function to each column
for column in data_mci .columns:
    data_mci [column] = data_mci [column].apply(extract_number)

for column in data_nc .columns:
    data_nc [column] = data_nc [column].apply(extract_number)

# Drop the identifier column if present
if data_mci.columns[0].lower() in ["id", "subject_id"]:
    data_mci = data_mci.drop(columns=data_mci.columns[0])
    data_nc = data_nc.drop(columns=data_nc.columns[0])

# Define the Wasserstein Distance test
results = []
for feature in data_mci.columns:
    # Extract feature values for MCI and NC groups
    mci_values = data_mci[feature].values
    nc_values = data_nc[feature].values

    # Calculate Wasserstein distance
    wasserstein_dist = wasserstein_distance(mci_values, nc_values)

    # Append results
    results.append((feature, wasserstein_dist))

# Convert results to a DataFrame for easy analysis
results_df = pd.DataFrame(results, columns=["Feature", "Wasserstein Distance"])

# Sort features by Wasserstein distance (higher indicates more dissimilarity)
results_df = results_df.sort_values(by="Wasserstein Distance", ascending=False)

# Save results to a CSV file
results_df.to_csv("wasserstein_distance_results.csv", index=False)

# Display the top features
print(f"Top Features by Wasserstein Distance:\n{results_df.head()}")

# Type 1 and Type 2 Error Analysis
def calculate_errors_and_power(data_mci, data_nc, num_permutations=1000, alpha=0.01):
    total_features = len(data_mci.columns)
    type_1_error_count = 0
    type_2_error_count = 0

    for feature in data_mci.columns:
        # Extract feature values
        mci_values = data_mci[feature].values
        nc_values = data_nc[feature].values

        # Observed Wasserstein distance
        observed_stat = wasserstein_distance(mci_values, nc_values)

        # Permutation testing
        all_values = np.concatenate([mci_values, nc_values])
        permuted_stats = []

        for _ in range(num_permutations):
            permuted = np.random.permutation(all_values)
            perm_mci = permuted[:len(mci_values)]
            perm_nc = permuted[len(mci_values):]
            perm_stat = wasserstein_distance(perm_mci, perm_nc)
            permuted_stats.append(perm_stat)

        # Calculate p-value
        p_value = np.mean(np.array(permuted_stats) >= observed_stat)

        # Type 1 error: Reject null when it is true
        if p_value < alpha:
            type_1_error_count += 1

        # Type 2 error: Fail to reject null when it is false
        if p_value >= alpha:
            type_2_error_count += 1

    # Calculate error rates and power
    type_1_error_rate = type_1_error_count / total_features
    type_2_error_rate = type_2_error_count / total_features
    power = 1 - type_2_error_rate

    return type_1_error_rate, type_2_error_rate, power

# Calculate Type 1 error, Type 2 error, and power
type_1_error_rate, type_2_error_rate, power = calculate_errors_and_power(data_mci, data_nc)

# Print summary
print(f"Type 1 Error Rate: {type_1_error_rate:.3f}")
print(f"Type 2 Error Rate: {type_2_error_rate:.3f}")
print(f"Power: {power:.3f}")


Top Features by Wasserstein Distance:
                           Feature  Wasserstein Distance
2                           Energy          2.767126e+09
15                     TotalEnergy          2.118771e+09
60  LargeAreaHighGrayLevelEmphasis          8.188021e+08
59               LargeAreaEmphasis          6.943076e+08
70                    ZoneVariance          6.941287e+08
Type 1 Error Rate: 0.966
Type 2 Error Rate: 0.034
Power: 0.966
